In [7]:
import sys
import json
import os
from category_encoders.leave_one_out import LeaveOneOutEncoder
import numpy as np
import joblib
from catboost.utils import read_cd
from sklearn.ensemble import RandomForestRegressor
from catboost import Pool

In [8]:
def create_dir(name):
    directory = os.path.dirname(name)
    if not os.path.exists(name):
        os.makedirs(name)

In [11]:
def generate_rf_ensemble_regression(dataset_name, n_splits=1,
                                    num_models=10, n_estimators = 1000, compress=3, 
                                    n_jobs=-1, max_depth=10):
                                    
    for fold in range(n_splits):
        
        # load and prepare data
        data_dir = os.path.join('datasets', dataset_name)
        full_train_file = os.path.join(data_dir, 'full_train')
        test_file = os.path.join(data_dir, 'test')
        cd_file = os.path.join(data_dir, 'pool.cd')

        full_train_pool = Pool(data=full_train_file, column_description=cd_file)
        test_pool = Pool(data=test_file, column_description=cd_file)

#         # make pools
#         X_train_all, y_train_all, X_train, y_train, X_validation, y_validation, X_test, y_test = make_train_val_test(X, y, index_train, index_test, fold)

        seed = 10 * fold # fix different starting random seeds for all folds
        for i in range(num_models):
            model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, 
                                          n_jobs=n_jobs, random_state=seed)
            seed += 1 # new seed for each ensemble element

            model.fit(full_train_pool.get_features(), full_train_pool.get_label()) 
            joblib.dump(model, "results/models/" + dataset_name + "_" + "rf" + "_f" + str(fold) + "_" + str(i), compress=compress)

In [13]:
mode = "regression_rf" #sys.argv[1]

if mode == "regression_rf":
    datasets = ["parkinsons"]
#     datasets = ["bostonHousing", "concrete", "energy", "kin8nm", 
#                 "naval-propulsion-plant", "power-plant", "protein-tertiary-structure",
#                 "wine-quality-red", "yacht", "YearPredictionMSD"] 

    create_dir("results/models")
    
    for name in datasets:
        print("dataset =", name)
    
        # Training all models
        print("training models...")
#         X, y, index_train, index_test, n_splits = load_regression_dataset(name)
        generate_rf_ensemble_regression(name, n_splits=1)

dataset = parkinsons
training models...


# Results

In [14]:
import numpy as np
from catboost import Pool, CatBoostRegressor
from scipy.stats import ttest_rel
from gbdt_uncertainty.assessment import prr_regression, nll_regression, ens_nll_regression, ood_detect
from gbdt_uncertainty.uncertainty import ensemble_uncertainties_regression
import math
import joblib
import sys
from collections import defaultdict

In [15]:
algorithms = ['sgb-fixed', 'sglb-fixed'] 

# for proper tables
convert_name = {"bostonHousing": "BostonH", "concrete": "Concrete", "energy": "Energy", 
                "kin8nm": "Kin8nm", "naval-propulsion-plant": "Naval-p", "power-plant": "Power-p",
                "protein-tertiary-structure": "Protein", "wine-quality-red": "Wine-qu", 
                "yacht": "Yacht", "YearPredictionMSD": "Year", 'parkinsons': 'Parkinsons'}

In [16]:
def calc_rmse(preds, target, raw=False):
    if raw:
        return (preds - target)**2 # for individual predictions
    return np.sqrt(np.mean((preds - target)**2))

def ens_rmse(target, preds, epsilon=1e-8, raw=False):
    means = preds[:, :, 0] 
    avg_mean = np.mean(means, axis=0) 
    if raw: # for individual predictions
        return calc_rmse(avg_mean, target, raw=True)
    return calc_rmse(avg_mean, target)

In [48]:
def load_and_predict(X, name, alg, fold, i):
    if alg == "rf":
        model = joblib.load("results/models/" + name + "_" + alg + "_f" + str(fold) + "_" + str(i))
        preds = model.predict(X)
        preds = np.array([(p, 1) for p in preds]) # 1 for unknown variance
    else:
        model = CatBoostRegressor()
        model.load_model("results/models/" + name + "_" + alg + "_f" + str(fold) + "_" + str(i)) 
        preds = model.predict(X)
    return preds, model
    
def predict(X, model, alg):
    preds = model.predict(X)
    if alg == "rf":
        preds = np.array([(p, 1) for p in preds])
    return preds
    
def rf_virtual_ensembles_predict(model, X, count=10):
    trees = model.estimators_
    num_trees = len(trees)
    ens_preds = []
    for i in range(count):
        indices = range(int(i*num_trees/count), int((i+1)*num_trees/count))
        all_preds = []
        for ind in indices:
            all_preds.append(trees[ind].predict(X))
        all_preds = np.array(all_preds)
        preds = np.mean(all_preds, axis=0)
        preds = np.array([(p, 1) for p in preds]) # 1 for unknown variance
        ens_preds.append(preds)
    ens_preds = np.array(ens_preds)

    return np.swapaxes(ens_preds, 0, 1)
    
def virtual_ensembles_load_and_predict(X, name, alg, fold, i, num_models=10):
    if alg == "rf":
        model = joblib.load("results/models/" + name + "_" + alg + "_f" + str(fold) + "_" + str(i))
        all_preds = rf_virtual_ensembles_predict(model, X)
    else:
        model = CatBoostRegressor()
        model.load_model("results/models/" + name + "_" + alg + "_f" + str(fold) + "_" + str(i)) 
        all_preds = model.virtual_ensembles_predict(X, prediction_type='VirtEnsembles', virtual_ensembles_count=num_models)
    return np.swapaxes(all_preds, 0, 1), model
  
def virtual_ensembles_predict(X, model, alg, num_models=10):
    if alg == "rf":
        all_preds = rf_virtual_ensembles_predict(model, X)
    else:
        all_preds = model.virtual_ensembles_predict(X, prediction_type='VirtEnsembles', virtual_ensembles_count=num_models)
    return np.swapaxes(all_preds, 0, 1)
    
def compute_significance(values_all, metric, minimize=True, raw=False):

    if raw:
        values_all = values_all[:, 0, :]

    values_mean = np.mean(values_all, axis=1) # mean wrt folds or elements
    
    if raw and metric == "rmse":
        values_mean = np.sqrt(values_mean)
    
    # choose best algorithm
    if minimize:
        best_idx = np.nanargmin(values_mean)
    else:
        best_idx = np.nanargmax(values_mean)
        
    textbf = {best_idx} # for all algorithms insignificantly different from the best one
    # compute statistical significance on test or wrt folds

    for idx in range(len(values_mean)):
        test = ttest_rel(values_all[best_idx], values_all[idx]) # paired t-test
        if test[1] > 0.05:
            textbf.add(idx)
            
    return values_mean, textbf

def compute_best(values, minimize=True):

    # choose best algorithm
    if minimize:
        best_idx = np.nanargmin(values)
    else:
        best_idx = np.nanargmax(values)
        
    textbf = {best_idx} 
    for idx in range(len(values)):
        if values[best_idx] == values[idx]: 
            textbf.add(idx)
            
    return textbf
    
def make_table_entry(values_all, metric, minimize=True, round=2, raw=True):
    
    num_values = len(values_all)
    
    values_mean, textbf = compute_significance(values_all, metric, minimize=minimize, raw=raw)

    # prepare all results in latex format

    table = ""

    for idx in range(num_values):
        if idx in textbf:
            table += "\\textbf{" + str(np.round(values_mean[idx], round)) + "} "
        else:    
            table += str(np.round(values_mean[idx], round)) + " "
        table += "& " 
            
    return table
            
def aggregate_results(name, modes = ["single", "ens", "virt"], 
                      algorithms = ['sgb-fixed', 'sglb-fixed'], num_models = 10, 
                      raw=False):
    
    n_splits=1
    if name != 'parkinsons':
        X, y, index_train, index_test, n_splits = load_regression_dataset(name)
    
    results = [] # metric values for all algorithms and all folds
    
    # for ood evaluation
    if name == 'parkinsons':
        ood_X_test = np.loadtxt("datasets/ood/" + name + '_rf')
        ood_X_test = ood_X_test[:-1, :-1]
    else:
        ood_X_test = np.loadtxt("datasets/ood/" + name)
    if name == "naval-propulsion-plant":
        ood_X_test = ood_X_test[:, :-1]
    
    ood_size = len(ood_X_test)
        
    for mode in modes:
        for alg in algorithms:
        
            values = defaultdict(lambda: []) # metric values for all folds for given algorithm

            for fold in range(n_splits):
                if name != 'parkinsons':
                    X_train_all, y_train_all, X_train, y_train, X_validation, y_validation, X_test, y_test = make_train_val_test(
                                                                                        X, y, index_train, index_test, fold)
                else:
                    ood_test_pool = Pool(data="datasets/ood/" + name + "_rf", column_description="datasets/"+name+"/pool.cd")
                    X_test, y_test = ood_test_pool.get_features(), ood_test_pool.get_label()
                    y_test = np.array(y_test).astype(np.float64)
                test_size = len(X_test)
                domain_labels = np.concatenate([np.zeros(test_size), np.ones(ood_size)])

                if mode == "single":
                    # use 0th model from ensemble as a single model
                    preds, model = load_and_predict(X_test, name, alg, fold, 0)

                    values["rmse"].append(calc_rmse(preds[:, 0], y_test, raw=raw))
                    values["nll"].append(nll_regression(y_test, preds[:, 0], preds[:, 1], raw=raw))
                    values["TU_prr"].append(prr_regression(y_test, preds[:, 0], preds[:, 1]))
                    values["KU_prr"].append(float("nan"))
                    values["KU_auc"].append(float("nan"))
                    
                    ood_preds = predict(ood_X_test, model, alg)
                    in_measure = preds[:, 1]
                    out_measure = ood_preds[:, 1]
                    values["TU_auc"].append(ood_detect(domain_labels, in_measure, out_measure, mode="ROC"))

                if mode == "ens":
                    all_preds = [] # predictions of all models in ensemble
                    all_preds_ood = []
                    
                    for i in range(num_models):
                        preds, model = load_and_predict(X_test, name, alg, fold, i)
                        all_preds.append(preds)
                        preds = predict(ood_X_test, model, alg)
                        all_preds_ood.append(preds)   
                    all_preds = np.array(all_preds)
                    
                    values["rmse"].append(ens_rmse(y_test, all_preds, raw=raw))
                    values["nll"].append(ens_nll_regression(y_test, all_preds, raw=raw)) 
                    
                    TU = ensemble_uncertainties_regression(np.swapaxes(all_preds, 0, 1))["tvar"]
                    KU = ensemble_uncertainties_regression(np.swapaxes(all_preds, 0, 1))["varm"]

                    mean_preds = np.mean(all_preds[:, :, 0], axis=0)

                    values["TU_prr"].append(prr_regression(y_test, mean_preds, TU))
                    values["KU_prr"].append(prr_regression(y_test, mean_preds, KU))
                    
                    all_preds_ood = np.array(all_preds_ood)
                    TU_ood = ensemble_uncertainties_regression(np.swapaxes(all_preds_ood, 0, 1))["tvar"]
                    KU_ood = ensemble_uncertainties_regression(np.swapaxes(all_preds_ood, 0, 1))["varm"]
                    values["TU_auc"].append(ood_detect(domain_labels, TU, TU_ood, mode="ROC"))
                    values["KU_auc"].append(ood_detect(domain_labels, KU, KU_ood, mode="ROC"))
                        
                if mode == "virt":
                    if alg in ["sgb", "sgb-fixed"]: # we do not evaluate virtual sgb model
                        continue
                    # generate virtual ensemble from 0th model
                    all_preds, model = virtual_ensembles_load_and_predict(X_test, name, alg, fold, 0)

                    values["rmse"].append(ens_rmse(y_test, all_preds, raw=raw))
                    values["nll"].append(ens_nll_regression(y_test, all_preds, raw=raw)) 
                    
                    TU = ensemble_uncertainties_regression(np.swapaxes(all_preds, 0, 1))["tvar"]
                    KU = ensemble_uncertainties_regression(np.swapaxes(all_preds, 0, 1))["varm"]
                    
                    mean_preds = np.mean(all_preds[:, :, 0], axis=0)

                    values["TU_prr"].append(prr_regression(y_test, mean_preds, TU))
                    values["KU_prr"].append(prr_regression(y_test, mean_preds, KU))
                    
                    all_preds_ood = virtual_ensembles_predict(ood_X_test, model, alg)
                    all_preds_ood = np.array(all_preds_ood)
                    
                    TU_ood = ensemble_uncertainties_regression(np.swapaxes(all_preds_ood, 0, 1))["tvar"]
                    KU_ood = ensemble_uncertainties_regression(np.swapaxes(all_preds_ood, 0, 1))["varm"]
                    
                    values["TU_auc"].append(ood_detect(domain_labels, TU, TU_ood, mode="ROC"))
                    values["KU_auc"].append(ood_detect(domain_labels, KU, KU_ood, mode="ROC"))

            if mode == "virt" and alg in ["sgb", "sgb-fixed"]: # we do not evaluate virtual sgb model
                continue
            
            results.append(values)

    return np.array(results)
    
def make_table_element(mean, textbf, idx):
    table = ""
    if np.isnan(mean[idx]):
        table += "--- & "
        return table
    if idx in textbf:
        table += "\\textbf{" + str(int(np.rint(mean[idx]))) + "} "
    else:    
        table += str(int(np.rint(mean[idx]))) + " "
    table += "& "
    return table

In [49]:
import warnings
warnings.filterwarnings("ignore")

table_type = "rf_prr_auc" #sys.argv[1]
datasets = ["parkinsons"]

if table_type == "rf_prr_auc":
    print("===Comparison with random forest, PRR and AUC-ROC===")
        
    for name in datasets:

        values = aggregate_results(name, algorithms=["sglb-fixed", "rf"], modes=["virt", "ens"], raw=False)
        
        prr_TU = np.array([values[i]["TU_prr"] for i in range(len(values))])
        prr_KU = np.array([values[i]["KU_prr"] for i in range(len(values))])
        prr = 100*np.concatenate((prr_TU, prr_KU), axis=0)

        mean_prr, textbf_prr = compute_significance(prr, "prr", minimize=False)
    
        auc_TU = np.array([values[i]["TU_auc"] for i in range(len(values))])
        auc_KU = np.array([values[i]["KU_auc"] for i in range(len(values))])
        auc = 100*np.concatenate((auc_TU, auc_KU), axis=0)
        mean_auc, textbf_auc = compute_significance(auc, "auc", minimize=False)

        num = len(auc_TU)
    
        table = "\multirow{2}{*} {" + convert_name[name] + "} & TU &"
        for idx in range(num):
            table += make_table_element(mean_prr, textbf_prr, idx)

        for idx in range(num):
            table += make_table_element(mean_auc, textbf_auc, idx)
            
        print(table.rstrip("& ") + " \\\\")
        
        table = " & KU & "
        for idx in range(num, 2*num):
            table += make_table_element(mean_prr, textbf_prr, idx)
            
        for idx in range(num, 2*num):
            table += make_table_element(mean_auc, textbf_auc, idx)
        print(table.rstrip("& ") + " \\\\")
        
        print("\midrule")

===Comparison with random forest, PRR and AUC-ROC===
\multirow{2}{*} {Parkinsons} & TU &-36 & \textbf{9} & -15 & -18 & \textbf{93} & 26 & 64 & 74 \\
 & KU & -46 & 9 & -13 & -18 & 80 & 26 & 59 & 74 \\
\midrule


In [50]:
table_type = "rf_rmse" #sys.argv[1]
datasets = ["parkinsons"]

if table_type == "rf_rmse": 

    print("===Comparison with random forest, RMSE===")
    for name in datasets:
        
        raw = False
        if name == "YearPredictionMSD":
            raw = True

        values = aggregate_results(name, algorithms=["sglb-fixed", "rf"], modes=["single", "ens"], raw=raw)
        
        table = convert_name[name] + " & "
        
        values_rmse = np.array([values[i]["rmse"] for i in range(len(values))])
        
        table += make_table_entry(values_rmse, "rmse", round=2, raw=raw)
        print(table.rstrip("& ") + " \\\\")

===Comparison with random forest, RMSE===
Parkinsons & \textbf{3.84} & 7.05 & 4.6 & 7.1 \\
